Добавить все необходимые библиотеки.

In [ ]:
pip install -U tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

Загрузка данных с диска и разделение на выборки обучающую(70 %), валидационную (15 %) и тестовую (15 %)..

In [ ]:
data_dir = '/content/drive/MyDrive/changed_data'

train_dataset = keras.preprocessing.image_dataset_from_directory(
    data_dir, subset='training', seed=42, validation_split=0.3, batch_size=100, image_size=(128, 128), shuffle=True
)

Found 17136 files belonging to 2 classes.
Using 11996 files for training.


In [ ]:
val_dataset = keras.preprocessing.image_dataset_from_directory(
    data_dir, subset='validation', seed=42, validation_split=0.3, batch_size=100, image_size=(128, 128), shuffle=True
)

Found 17136 files belonging to 2 classes.
Using 5140 files for validation.


In [ ]:
test_dataset = val_dataset.take(2570) 
val_dataset = val_dataset.skip(2570)

In [ ]:
input_shape = (128, 128, 3)
num_classes = 2

Регулирование гиперпараметров для нейронной сети типа ViT.

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
# В одном пакете будет
batch_size = 100
# Количество эпох
num_epochs = 20
# Размер одного патча взятого из изображения
patch_size = 16 
# Общее колиество патчей
num_patches = (128 // patch_size) ** 2

# Гиперпараметры для нейронной сети
projection_dim = 64
num_heads = 4
# Размер слоёв трасформера
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

Многослойный персептрон.

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = keras.layers.Dense(units, activation=tf.nn.gelu)(x)
        x = keras.layers.Dropout(dropout_rate)(x)
    return x

Создание патчей из одного изображения

In [ ]:
class Patches(keras.layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


Кодировщик патчей.

In [ ]:
class PatchEncoder(keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = keras.layers.Dense(units=projection_dim)
        self.position_embedding = keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

Создание модели данных на основе нейронной сети "Трансформер".

In [ ]:
def create_vit_classifier():
    inputs = keras.layers.Input(shape=input_shape)
    # Создание патчей
    patches = Patches(patch_size)(inputs)
    # Кодировка патчей
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Создание множества слоёв
    for _ in range(transformer_layers):
        # Нормализация слоя
        x1 = keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        x2 = keras.layers.Add()([attention_output, encoded_patches])
        # Нормализация слоя 
        x3 = keras.layers.LayerNormalization(epsilon=1e-6)(x2)
        # Многослойный персептрон
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        encoded_patches = keras.layers.Add()([x3, x2])

    # Создание тензора
    representation = keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = keras.layers.Flatten()(representation)
    representation = keras.layers.Dropout(0.5)(representation)
    # Добавление многослойного пресептрона
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Классификация вывода
    logits = keras.layers.Dense(num_classes)(features)
    # Создание модели keras
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


Обучение модели данных.

In [ ]:
# Функция обучения модели данных
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    # Компиляция
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )

    # Сохранение модели данных
    checkpoint_filepath = "/content/drive/MyDrive/NN_models/ViT_model"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor='accuracy',
        save_best_only=True,
    )

    # обучение
    history = model.fit(
        train_dataset,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=val_dataset,
        callbacks=[checkpoint_callback])

    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(test_dataset)
    print(f"Точность тестов: {round(accuracy * 100, 2)}%")

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)


Epoch 1/20
120/120 [==============================] - ETA: 0s - loss: 1.0690 - accuracy: 0.5506

INFO:tensorflow:Assets written to: /content/drive/MyDrive/NN_models/ViT_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/NN_models/ViT_model/assets


120/120 [==============================] - 321s 3s/step - loss: 1.0690 - accuracy: 0.5506
Epoch 2/20
120/120 [==============================] - ETA: 0s - loss: 0.6859 - accuracy: 0.5843

INFO:tensorflow:Assets written to: /content/drive/MyDrive/NN_models/ViT_model/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/NN_models/ViT_model/assets


120/120 [==============================] - 322s 3s/step - loss: 0.5211 - accuracy: 0.7084
Epoch 20/20
120/120 [==============================] - ETA: 0s - loss: 0.5149 - accuracy: 0.7127

INFO:tensorflow:Assets written to: /content/drive/MyDrive/NN_models/ViT_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/NN_models/ViT_model/assets


52/52 [==============================] - 179s 3s/step - loss: 0.5361 - accuracy: 0.6875
Точность тестов: 68.75%
